#### Lab 06: Temporal analysis

##### Section: 52740

##### Group #: 13

##### Student: Arianna Zhao
##### UT EID:  axz83

##### Student: Raul Denicolas
##### UT EID:  rd32483

##### Student: Nibhan Akbar
##### UT EID:  na26287

##### Student: Victor Fonseca
##### UT EID:  VRF262

We encounter time series data in pretty much every domain, from finance to weather, from public health to renewable energies. Visualizations of temporal data may represent recorded observations from the past and/or predicted developments for the future, which is why visual representations of temporal data are so important and interesting. Especially, in the context of the ongoing climate and covid crises we encounter many time series visualizations.

This tutorial is structured into three basic phases: 1. Prepare, 2. Process, and 3. Present.


#### 1. Prepare

Before we are able to do anything, we need to include the libraries that we are working with (as always):

In [ ]:
import pandas as pd
import altair as alt
import scipy.signal # for the LOESS fitting

##### Parse dates and times

In its most basic form, time series data contain a quantitative measure that changes over time. To reference a time point we use [Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html) of Pandas as the data type for temporal entities.

With **`to_datetime()`** you can create a Timestamp with a string containing a date and/or time. Pandas can infer the date and time from various date/time strings. Let's start with the present:


In [ ]:
pd.to_datetime("now")

<ipython-input-89-9c9d722949e0>:1: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  pd.to_datetime("now")


Timestamp('2023-10-14 03:52:25.297458')

We can pass a range of date formats and Pandas will guess which numbers refer to years, months, days, hours, etc.:

In [ ]:
pd.to_datetime('2023-10-13 3pm')

Timestamp('2023-10-13 15:00:00')

When expressing dates and times in written language, there is an ambiguity between the order of different entities. The most frequent ambiguity concerns the order of days and months, as they are typyically both expressed in double-digit numbers, unlike years that tend to be expressed with four digits. However, date conventions vary across the world.  For example, the following date might be interpreted differently depending on the country; it may refer to Saint Nicholas Day in 1929 or Anne Frank's birthday:

In [ ]:
pd.to_datetime('12.6.1929')

Timestamp('1929-12-06 00:00:00')

To clarify towards Pandas that the first number refers to the day, you can add the parameter **`dayfirst`**:

In [ ]:
pd.to_datetime('12.6.1929', dayfirst=True)

Timestamp('1929-06-12 00:00:00')

The method `to_datetime()` can also handle an array of date strings; it will return a `DatetimeIndex`, which is crucial for temporal indexing with Pandas.

In [ ]:
sessions=["2.4.2020", "9.4.2020", "16.4.2020", "23.4.2020", "7.5.2020", "14.5.2020", "28.5.2020", "4.6.2020", "11.6.2020", "25.6.2020", "2.7.2020", "9.7.2020"]
pd.to_datetime(sessions, dayfirst=True)

DatetimeIndex(['2020-04-02', '2020-04-09', '2020-04-16', '2020-04-23',
               '2020-05-07', '2020-05-14', '2020-05-28', '2020-06-04',
               '2020-06-11', '2020-06-25', '2020-07-02', '2020-07-09'],
              dtype='datetime64[ns]', freq=None)

If you want to make extra sure that the date/time string is parsed correctly and quickly, you can pass a fixed **`format`** for the date/time strings to be parsed:

In [ ]:
pd.to_datetime('2020-05-07', format="%Y-%m-%d")

Timestamp('2020-05-07 00:00:00')

*see [list of format codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)!*

##### Load time series data

In this tutorial we will be analyzing energy use in Germany from 2015 until 2018.

First we will load the freely accessible data from the [OPSD project](https://open-power-system-data.org). `read_csv()` has a convenient feature, which lets you specify the column containing date/time information.

In [ ]:
# read and create data frame and parse the dates

df = pd.read_csv('german_energy_data.csv', parse_dates=['datetime'])

###### Do Sanity Checks

In [ ]:
df.shape

(35064, 4)

In [ ]:
df.head()

,datetime,load,solar,wind
0,2014-12-31 23:00:00+00:00,41987,0,8644
1,2015-01-01 00:00:00+00:00,40665,0,8814
2,2015-01-01 01:00:00+00:00,39510,0,9054
3,2015-01-01 02:00:00+00:00,38537,0,9070
4,2015-01-01 03:00:00+00:00,38153,0,9164


In [ ]:
df.tail()

,datetime,load,solar,wind
35059,2018-12-31 18:00:00+00:00,52829,0,16438
35060,2018-12-31 19:00:00+00:00,49667,0,17587
35061,2018-12-31 20:00:00+00:00,47369,0,18872
35062,2018-12-31 21:00:00+00:00,46890,0,20270
35063,2018-12-31 22:00:00+00:00,45079,0,21405


In [ ]:
df.tail()

,datetime,load,solar,wind
35059,2018-12-31 18:00:00+00:00,52829,0,16438
35060,2018-12-31 19:00:00+00:00,49667,0,17587
35061,2018-12-31 20:00:00+00:00,47369,0,18872
35062,2018-12-31 21:00:00+00:00,46890,0,20270
35063,2018-12-31 22:00:00+00:00,45079,0,21405


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  35064 non-null  datetime64[ns, UTC]
 1   load      35064 non-null  int64              
 2   solar     35064 non-null  int64              
 3   wind      35064 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(3)
memory usage: 1.1 MB


In [ ]:
df.describe()

,load,solar,wind
count,35064.000000,35064.000000,35064.000000
mean,55923.248546,4180.542665,10429.418863
std,10081.865874,6412.483093,8190.691684
min,31455.000000,0.000000,135.000000
25%,47505.000000,0.000000,4096.000000
50%,55628.000000,122.000000,8084.500000
75%,64862.250000,6619.250000,14700.000000
max,79063.000000,28955.000000,44629.000000



The time parsing resulted in one hour to be included from 2014. This is because this dataset covers the years 2015-2018 according to German time, while the generic UTC time is one hour 'behind'. To avoid having this stray hour in the old year, we revert the timestamps back to Germany's timezone:

In [ ]:
df["datetime"] = df["datetime"].dt.tz_convert("Europe/Berlin")
df.head()

,datetime,load,solar,wind
0,2015-01-01 00:00:00+01:00,41987,0,8644
1,2015-01-01 01:00:00+01:00,40665,0,8814
2,2015-01-01 02:00:00+01:00,39510,0,9054
3,2015-01-01 03:00:00+01:00,38537,0,9070
4,2015-01-01 04:00:00+01:00,38153,0,9164


Did you notice that solar energy appears to be zero at the beginning and at the end of the DataFrame? Take a look at the time and you'll know why.

In order to get a random data `sample()` there is a method of the same name, which gives us a random set of rows:

In [ ]:
df.sample(10)

,datetime,load,solar,wind
12663,2016-06-11 16:00:00+02:00,48808,11603,936
17797,2017-01-11 13:00:00+01:00,76944,1382,31168
19459,2017-03-21 19:00:00+01:00,67084,0,12890
20776,2017-05-15 17:00:00+02:00,61074,12267,2668
14202,2016-08-14 19:00:00+02:00,45825,2018,4272
33265,2018-10-18 02:00:00+02:00,46673,0,3200
25463,2017-11-26 23:00:00+01:00,52542,0,24618
28687,2018-04-10 08:00:00+02:00,67176,6277,13553
34202,2018-11-26 02:00:00+01:00,46743,0,14113
27928,2018-03-09 16:00:00+01:00,66860,3046,6170


Now, let's take a look at the data types and values contained in our DataFrame:

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype                        
---  ------    --------------  -----                        
 0   datetime  35064 non-null  datetime64[ns, Europe/Berlin]
 1   load      35064 non-null  int64                        
 2   solar     35064 non-null  int64                        
 3   wind      35064 non-null  int64                        
dtypes: datetime64[ns, Europe/Berlin](1), int64(3)
memory usage: 1.1 MB


The values in the columns `load`, `solar` and `wind` are provided in the unit megawatt (MW) as integers.

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame:

In [ ]:
df = df.set_index("datetime")

When we run `df.info()` again, you will see that the DataFrame now has a `DatetimeIndex`:

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35064 entries, 2015-01-01 00:00:00+01:00 to 2018-12-31 23:00:00+01:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   load    35064 non-null  int64
 1   solar   35064 non-null  int64
 2   wind    35064 non-null  int64
dtypes: int64(3)
memory usage: 1.1 MB


The `DatetimeIndex` provides a few handy methods to extract temporal units such as months, days, week of the year, etc.:

In [ ]:
df.index.year.unique()

Int64Index([2015, 2016, 2017, 2018], dtype='int64', name='datetime')

*Look here for syntax to extract any other [temporal attributes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html)*

#### 2. Process

The dataset contains over 35000 rows each of which contains data on overall energy load and renewable energy production. To make sense of all this data, we need to process the time series data into various chunks and sizes!


### Query time points and spans

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

One useful function of a temporal index, is its querying function. We can quickly extract the rows for a given time point or period.

Let's get the data for the day with the most daylight of 2017, i.e., the summer solstice or midsummer:

In [ ]:
df.loc["2017-06-21"]

,load,solar,wind
datetime,,,
2017-06-21 00:00:00+02:00,48260,0,7394
2017-06-21 01:00:00+02:00,46107,0,6460
2017-06-21 02:00:00+02:00,44959,0,5894
2017-06-21 03:00:00+02:00,45136,0,5472
2017-06-21 04:00:00+02:00,45486,3,5008
2017-06-21 05:00:00+02:00,47746,303,4602
2017-06-21 06:00:00+02:00,55753,2095,3688
2017-06-21 07:00:00+02:00,62259,5926,2113
2017-06-21 08:00:00+02:00,65541,11142,1482


Above query is an example of partial-string indexing: while our `DateTime` column actually contains time information as well, you can query it quickly (!) with just the date, or even a shorter query:

In [ ]:
df.loc["2017-06"]

,load,solar,wind
datetime,,,
2017-06-01 00:00:00+02:00,46372,0,7902
2017-06-01 01:00:00+02:00,44690,0,7233
2017-06-01 02:00:00+02:00,43592,0,6642
2017-06-01 03:00:00+02:00,43745,0,6174
2017-06-01 04:00:00+02:00,44374,3,6160
...,...,...,...
2017-06-30 19:00:00+02:00,59717,3010,12646
2017-06-30 20:00:00+02:00,56061,876,10976
2017-06-30 21:00:00+02:00,53473,40,9854


You can also query a time period. Do you remember the storm Xavier that hit Germany in early October 2017? Let's retrieve the data around this time:

In [ ]:
df.loc["2017-10-04":"2017-10-06"]

,load,solar,wind
datetime,,,
2017-10-04 00:00:00+02:00,43116,0,19096
2017-10-04 01:00:00+02:00,41615,0,19163
2017-10-04 02:00:00+02:00,41348,0,19162
2017-10-04 03:00:00+02:00,41851,0,19020
2017-10-04 04:00:00+02:00,43778,0,19456
...,...,...,...
2017-10-06 19:00:00+02:00,65038,0,21984
2017-10-06 20:00:00+02:00,60991,0,21764
2017-10-06 21:00:00+02:00,57189,0,21500


##### Aggregate values along time

The data we retrieved from OPSD, comes in the granularity of hours. To better understand the data it can be useful to reduce the data resolution and consider, for example, the total energy used/produced everyday or the daily averages over the course of entire years.

The DataFrame's `resample()` [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html) method provides a concise and quick way of aggregating temporally indexed data along time units. Here we create a DataFrame with summed up values for each year aggregated from the original dataset:

In [ ]:
sums = df.resample("Y").sum()
sums

,load,solar,wind
datetime,,,
2015-12-31 00:00:00+01:00,477924124,34907138,77468981
2016-12-31 00:00:00+01:00,481084453,34562824,77001342
2017-12-31 00:00:00+01:00,493271535,35882643,102651209
2018-12-31 00:00:00+01:00,508612675,41233943,108575611


The resample operations can also be carried out one after another. For example, we might want to know how weekly energy use/production varies between the quarters:

In [ ]:
# first we create the sums per week
weekly_sums = df.resample("W").sum()
# then we generate the weekly means for each quarter
quarterly_means = weekly_sums.resample("Q").mean()
# for readability we'll revert the values back to integers
quarterly_means.astype(int)

,load,solar,wind
datetime,,,
2015-03-31 00:00:00+02:00,9236885,358245,1563098
2015-06-30 00:00:00+02:00,8666798,1013041,1195968
2015-09-30 00:00:00+02:00,8979745,978020,1152864
2015-12-31 00:00:00+01:00,9528732,326486,1952230
2016-03-31 00:00:00+02:00,9457773,314465,1923228
2016-06-30 00:00:00+02:00,8847897,995637,1234926
2016-09-30 00:00:00+02:00,8842918,1046984,990449
2016-12-31 00:00:00+01:00,9664017,296777,1691937
2017-03-31 00:00:00+02:00,9916792,376473,2178930


##### 3. Present

Enough data processing. It's time for visualization!

#### Time spans

One of the first time visualizations was [*A Chart of Biography*](https://en.wikipedia.org/wiki/A_Chart_of_Biography) (1765) by Joseph Priestley. Let's create a similar visualization of the US presidencies since World War II. First we load the CSV file with `pd.read_csv()`:

In [ ]:
presidents = pd.read_csv('us_presidents.csv')

In [ ]:
presidents.shape

(14, 4)

In [ ]:
presidents.head()

,name,start,end,party
0,Harry S. Truman,1945,1953,Democratic
1,Dwight D. Eisenhower,1953,1961,Republican
2,John F. Kennedy,1961,1963,Democratic
3,Lyndon B. Johnson,1963,1969,Democratic
4,Richard Nixon,1969,1974,Republican


In [ ]:
presidents.tail()

,name,start,end,party
9,Bill Clinton,1993,2001,Democratic
10,George W. Bush,2001,2009,Republican
11,Barack Obama,2009,2017,Democratic
12,Donald Trump,2017,2021,Republican
13,Joe Biden,2021,2025,Democratic


In [ ]:
presidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    14 non-null     object
 1   start   14 non-null     int64 
 2   end     14 non-null     int64 
 3   party   14 non-null     object
dtypes: int64(2), object(2)
memory usage: 576.0+ bytes


The start and end times of the presidencies are given as four-digit integers, i.e., years. To convey towards Pandas and Altair that the start and end columns are actually dates, we need to parse them using the `to_datetime()` method:

In [ ]:
presidents['start'] = pd.to_datetime(presidents['start'], format="%Y")
presidents['end'] = pd.to_datetime(presidents['end'], format="%Y")

The following chart consists of two parts: `bars` and `labels`. The former will be the main bar chart representing the time spans of the presidencies, and the latter will add the presidents' names. This way we can position the labels right next to the bars, much nicer!

In [ ]:
# we add height as a parameter to make the time spans appear less clunky
bars = alt.Chart(presidents).mark_bar(height=5).encode(
    # this time we need two columns for x-position
    x='start', x2='end',
    # we sort by start dates and hide the axis, as names are added below
    y=alt.Y('name', sort='x', axis=None),
    color="party"
)

# the labels are added to the barchart using the mark_text command
labels = bars.mark_text(align='right', dx=-5).encode(text='name')

# both charts integrated into a layered chart through the magic of a plus sign
(bars + labels).properties(width=800, height=400)

alt.LayerChart(...)

##### Overall trends

Next, we are going to return to the energy time series data that we prepared above. Remember that the original dataset has an hourly resolution resulting in too many data points to visualize at once. Altair itself handles at most 5000 data rows.

To reduce the dataset into a manageable size, we  will create daily sums with the `resample()` method of the DateTimeIndex:

In [ ]:
days = df.resample("D").sum()

To visualize the contents of the DataFrame with Altair, we turn the date/time information in the index into its own column `datetime` again and transform the DataFrame from [wide to long form](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data) using the `melt()` method. We pass the name of the `datetime` column to the melt method so that it is treated as an index variable. To keep it short, these two steps `reset_index()` and `melt("datetime")` are done in one go right when we create the `Chart()`, which is what we are going to the in the rest of the tutorial.

Now let's start with a scatterplot visualization of the daily data:

In [ ]:
# Break down the long form of the data so we can see that it works
days.reset_index().melt("datetime",var_name='Type')

,datetime,Type,value
0,2015-01-01 00:00:00+01:00,load,1061334
1,2015-01-02 00:00:00+01:00,load,1267901
2,2015-01-03 00:00:00+01:00,load,1124883
3,2015-01-04 00:00:00+01:00,load,1072455
4,2015-01-05 00:00:00+01:00,load,1304357
...,...,...,...
4378,2018-12-27 00:00:00+01:00,wind,286302
4379,2018-12-28 00:00:00+01:00,wind,227879
4380,2018-12-29 00:00:00+01:00,wind,511284
4381,2018-12-30 00:00:00+01:00,wind,590481


In [ ]:
alt.Chart(days.reset_index().melt("datetime")).mark_circle().encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=800, height=400)

alt.Chart(...)

With this our eyes can already see several patterns going on. Some are more dictinct than others. Interestingly, the load data (blue) seems to have two separate, but parallel curves…

✏️ *There is a lot of overplotting going on. Reduce the `size` and `opacity` of all dots, by passing these as parameters to `mark_circle`!*

Next, we are going to connect the dots and create a line chart form this data. So basically the same code as above, except we're now using `mark_line()` instead of `mark_circle`:

In [ ]:
alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1).encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=800, height=400)

alt.Chart(...)

This chart already shows a lot: we can see weekly patterns—the jittery up and down—in the load (blue) and the seasonal patterns in the form of broad waves in the load and solar curves (blue and yellow). Right around the turn of the year we see a drop of energy load.

While above line chart is truthful to the local fluctuations, it makes it hard to actually grasp the up and down over the course of months and years. Let's change the sampling from days to months to examine the overall patterns in the data.

In [ ]:
months = df.resample("M").sum()

# to make the line a bit smoother, we include an interpolate parameter
alt.Chart(months.reset_index().melt("datetime")).mark_line(opacity=0.75, interpolate="basis").encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=800, height=400)

alt.Chart(...)

What do you think? The fine-grained jitter is now gone and we might have lost too much detail. In fact, first downsampling the data and then including an interpolation is maybe giving it too much of a treatment (like overusing Photoshop's blur function). For example, the load minima around the turns of the year are not visible as the overall energy load in the winter months appears larger. One the other hand, the solar curve has become an almost perfect sine wave, probably relating to the position of the sun across the year.

💡 *Play with different granularities in he `resample()` step. For example, use **W** for week or **Q** for quarter!*

One way to integrate the local and global patterns is to create a layered graph, as we have already done with the presidents' names above. This time we are combining a line chart of the days with a line chart of monthly averages.

Note how we ensure that both DataFrames fit the same data scale. As we're looking at energy values per day, we first generate the daily sums and on this basis the monthly means:

In [ ]:
days = df.resample("D").sum()
months = days.resample("M").mean()

Next we create the line charts and combine the two again with the **+** operator:

In [ ]:
chart1 = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1, opacity=0.25).encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=800, height=400)

chart2 = alt.Chart(months.reset_index().melt("datetime")).mark_line(interpolate="basis", opacity=1).encode(
    x='datetime',
    y='value',
    color='variable',
)

chart1 + chart2

alt.LayerChart(...)

With this view we already get a good sense of the overall time patterns, while still seeing some of the particular variations.

💡 *You can add the `.interactive()` directive to one of these charts to make them zoomable!*

### Rolling windows

While the `resample()` method takes a broad brush and results in a reduced dataset and a chart with smooth curves, `rolling()` offers an alternative way of smoothing out local outliers without actually reducing the resolution of the dataset.

The first parameter determines the window size, by positioning the window at the `center` values are considered in both directions of the current date/time, and `win_type` determines how the values across the window are weighted; with the `triang` option the values further away contribute less:

In [ ]:
rolling = days.rolling(60, center=True, win_type="triang").mean()

chart1 = alt.Chart(rolling.reset_index().melt("datetime")).mark_line(strokeWidth=1.5, opacity=1).encode(
    x='datetime', y='value', color='variable',
).properties(width=800, height=400)

# same as the two charts in previous code cell, except more transparent
chart2 = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1, opacity=0.1).encode(
    x='datetime', y='value', color='variable',
)

chart3 = alt.Chart(months.reset_index().melt("datetime")).mark_line(interpolate="basis", opacity=.25).encode(
    x='datetime', y='value', color='variable',
)

chart1 + chart2 + chart3

alt.LayerChart(...)

Above you see the two lines for daily sums and monthly averages from the previous cell (slightly more transparent), on top of which you can see the time curve generated with a rolling window. It is quite apparent that this curve still features more pronounced dips around the end-of-year periods and elsewhere.


💡 *Play around with different window sizes and other parameters in the first line in above cell!*

### Detailed views

Some trends seem to pan out at much lower scales, which require higher levels of detail. For example, at the beginning of 2018 we see a peak in the wind energy production. In the winter of 2017/2018 Germany and most of Europe actually witnessed several storms. And so it also happened in the first days of 2018, when [Burglind (a.k.a. Eleanor)](https://en.wikipedia.org/wiki/Storm_Eleanor_(2018)) passed by. Let's take a closer look at the first week of January 2018. For this we extract a subset of the DataFrame using our tried and tested `loc[]`, which lets us select specific time spans:

In [ ]:
burglind = df.loc["2018-01-01":"2018-01-07"]

alt.Chart(burglind.reset_index().melt("datetime")).mark_line().encode(
    x='datetime',
    y='value',
    color='variable'
).properties(width=800, height=400)

alt.Chart(...)

### High-level bars

In contrast to the detailed close-up, we might also want to look at much more general patterns. Let's say we are curious about the total production of renewable energy over the four years in the dataset.

For this we `resample` the original data from hourly data resolution into summed up values per year:

In [ ]:
years = df.resample("Y").sum()

Next we create a new column, which contains the summed up values of solar and wind production.

In [ ]:
years["renewable"] = years["solar"]+years["wind"]

We `drop` the other value columns:

In [ ]:
renewable = years.drop(columns=["load", "solar", "wind"])

Extract the years and create a specific column:

In [ ]:
renewable["year"] = years.index.year
renewable

,renewable,year
datetime,,
2015-12-31 00:00:00+01:00,112376119,2015
2016-12-31 00:00:00+01:00,111564166,2016
2017-12-31 00:00:00+01:00,138533852,2017
2018-12-31 00:00:00+01:00,149809554,2018


And we remove the datetime index, and this time discard it (i.e., `drop=True`), as we have the `year` column already:

In [ ]:
renewable = renewable.reset_index(drop=True)
renewable

,renewable,year
0,112376119,2015
1,111564166,2016
2,138533852,2017
3,149809554,2018


Now we can create a simple barchart from this:

In [ ]:
alt.Chart(renewable).mark_bar(width=30, fill="purple").encode(
    x='year:O',
    y='renewable:Q'
).properties(width=300, height=300)

alt.Chart(...)

##### Cyclical patterns

Previously, we did notice a few recurring patterns in the energy load. One pattern might be due to the weekly work-and-leasure patterns.

So let's take a closer look at the average energy `load` over the course of a week:


In [ ]:
# remove solar and wind columns
weekdays = df.drop(columns=["solar", "wind"])

# add a column for weekdays
weekdays["weekday"] = weekdays.index.weekday

This time we do not use `resample()` to change the data granularity, but `groupby()` to study recurring data patterns, because we are actually interested in the energy load for each *generic* weekday across the entire dataset:

In [ ]:
weekdays = weekdays.groupby("weekday").mean()

alt.Chart(weekdays.reset_index()).mark_bar(width=20).encode(
    x='weekday:O',
    y='load:Q'
).properties(width=300, height=300)

alt.Chart(...)

It's quite apparent that energy load is lower during weekends.

##### Interactive selection

If you want it all: A high-level overview and detailed close-ups, you might need two coordinated views:

In [ ]:
weeks = df.resample("W").sum()
days = df.resample("D").sum()

brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(weeks.reset_index().melt("datetime")).mark_area(interpolate="basis").encode(
    x = alt.X('datetime:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=800, height=50).add_selection(brush)

lower = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1).encode(
    x = alt.X('datetime:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=800, height=300)

upper & lower

alt.VConcatChart(...)

Note: The small stacked graph above the main chart is interactive, you can adjust the viewport of the line graph by dragging a time span with it.

#### Your turn

Apply your understanding of the above topics and answer the following 6 questions

In [ ]:
# 1 - Look at the documentation to translate the string to datetime using the correct format for VE day
# pd.to_datetime('8.5.1945 23:01', format="")
pd.to_datetime('8.5.1945 23:01', format="%m.%d.%Y %H:%M")

Timestamp('1945-08-05 23:01:00')

In [ ]:
# 2 - append code to `df.index` below that shows what time zone the data is in
# df.index
df.index.dtype

datetime64[ns, Europe/Berlin]

In [ ]:
# 3 - Use date slicing/indexing to find the mean of the solar energy from
# 21 June 2018 and the mean of the solar energy from 20 December 2018
# df.loc[
start = pd.to_datetime('2018-06-21 00:00:00+01:00')
end = pd.to_datetime('2018-06-21 23:00:00+01:00')
df_21june18 = df.loc[(df.index >= start) & (df.index <= end)]
df_21june18['solar'].mean()

7543.791666666667

In [ ]:
# df.loc[
start = pd.to_datetime('2018-12-20 00:00:00+01:00')
end = pd.to_datetime('2018-12-20 23:00:00+01:00')
df_20dec18= df.loc[(df.index >= start) & (df.index <= end)]
df_20dec18['solar'].mean()

767.4583333333334

In [ ]:
# 4 - How did daily sum energy use/production change over the years? (calculate the daily sum, then the year average)
# and show the resulting dataframe
df['Total Energy'] = df['solar'] + df['wind']
df.tail()


,load,solar,wind,Total Energy
datetime,,,,
2018-12-31 19:00:00+01:00,52829,0,16438,16438
2018-12-31 20:00:00+01:00,49667,0,17587,17587
2018-12-31 21:00:00+01:00,47369,0,18872,18872
2018-12-31 22:00:00+01:00,46890,0,20270,20270
2018-12-31 23:00:00+01:00,45079,0,21405,21405


In [ ]:
# year average use for 2015
start = pd.to_datetime('2015-01-01 00:00:00+01:00')
end = pd.to_datetime('2015-12-31 23:00:00+01:00')
df_2015= df.loc[(df.index >= start) & (df.index <= end)]
df_2015['load'].mean()

54557.54840182648

In [ ]:
# year average production for 2015
start = pd.to_datetime('2015-01-01 00:00:00+01:00')
end = pd.to_datetime('2015-12-31 23:00:00+01:00')
df_2015= df.loc[(df.index >= start) & (df.index <= end)]
df_2015['Total Energy'].mean()

12828.324086757992

In [ ]:
# year average use for 2016
start = pd.to_datetime('2016-01-01 00:00:00+01:00')
end = pd.to_datetime('2016-12-31 23:00:00+01:00')
df_2016= df.loc[(df.index >= start) & (df.index <= end)]
df_2016['load'].mean()

54768.266507285975

In [ ]:
# year average production for 2016
start = pd.to_datetime('2016-01-01 00:00:00+01:00')
end = pd.to_datetime('2016-12-31 23:00:00+01:00')
df_2016= df.loc[(df.index >= start) & (df.index <= end)]
df_2016['Total Energy'].mean()

12700.838570127504

In [ ]:
# year average use for 2017
start = pd.to_datetime('2017-01-01 00:00:00+01:00')
end = pd.to_datetime('2017-12-31 23:00:00+01:00')
df_2017= df.loc[(df.index >= start) & (df.index <= end)]
df_2017['load'].mean()

56309.53595890411

In [ ]:
# year average production for 2017
start = pd.to_datetime('2017-01-01 00:00:00+01:00')
end = pd.to_datetime('2017-12-31 23:00:00+01:00')
df_2017= df.loc[(df.index >= start) & (df.index <= end)]
df_2017['Total Energy'].mean()

15814.366666666667

In [ ]:
# year average use for 2018
start = pd.to_datetime('2017-01-01 00:00:00+01:00')
end = pd.to_datetime('2017-12-31 23:00:00+01:00')
df_2018= df.loc[(df.index >= start) & (df.index <= end)]
df_2018['load'].mean()

56309.53595890411

In [ ]:
# year average production for 2018
start = pd.to_datetime('2017-01-01 00:00:00+01:00')
end = pd.to_datetime('2017-12-31 23:00:00+01:00')
df_2018= df.loc[(df.index >= start) & (df.index <= end)]
df_2018['Total Energy'].mean()

15814.366666666667

In [ ]:
# 5 - Create a horizontal bar chart showing total renewable energy over the years
years = pd.DataFrame({"Year": [2015, 2016, 2017, 2018],"Mean Value": [df_2015["Total Energy"].mean(),df_2016["Total Energy"].mean(),df_2017["Total Energy"].mean(),df_2018["Total Energy"].mean()]})

hist = alt.Chart(years).mark_bar().encode(
    x='Mean Value:Q',
    y='Year:N',
    color='Year:N'
)
hist

alt.Chart(...)

In [ ]:
# 6 - Show the wind energy for the month of Dec 2018 along with the Savitzky–Golay filter line overlayed on top.
# as a line chart.  Set your window for one week of data. Play around with the polynomial order until you
# get something you like.
dec_2018=df["2018-12-01": "2018-12-31"]
wind=scipy.signal.savgol_filter(dec_2018["wind"], window_length=7, polyorder=2)
dec_2018["wind_filtered"]=wind
#dec_2018
base_df = pd.melt(
    dec_2018.reset_index(),
    id_vars='datetime',
    value_vars=['wind', 'wind_filtered'])

layered_chart = alt.Chart(base_df).mark_line().encode(
    x='datetime:T',
    y='value:Q',
    color='variable:N'
)

layered_chart

<ipython-input-152-1b88d616d1e5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dec_2018["wind_filtered"]=wind


alt.Chart(...)

## Sources

Tutorials & Examples
- [​​​​Tutorial: Time Series Analysis with Pandas by Jennifer Walker](https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/)
- [Altair Interval Selection Example](https://altair-viz.github.io/gallery/interval_selection.html)

Documentation: Pandas
- [Time series / date functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)
- [Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html)
- [DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html)
- [Time-aware rolling vs. resampling](https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#time-aware-rolling-vs-resampling)

